In [1]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import matplotlib.pyplot as plt

In [2]:
import os
path = os.getcwd()
samples = pd.read_csv(path + "./news2000.csv")

In [3]:
# token
from konlpy.tag import Okt
okt = Okt()
stop_words=set(['은','는','이','가','하','아','것','들','의','있','되','수','보','주','등','한'])

def preprocess(sentence):
    word_to_id = {}
    id_to_word = {}

    # 토큰화
    token = [okt.morphs(words) for words in sentence if isinstance(words, str)]

    # 불용어
    for i, words in enumerate(token):
        token[i] = [word for word in words if not word in stop_words]
    
    # word2dict
    for words in token:
        for word in words:
            if  word not in word_to_id.keys():
                new_id = len(word_to_id)
                word_to_id[word] = new_id
                id_to_word[new_id] = word 
    
    # 말뭉치
    corpus_ar = []
    for words in token:
        for word in words:
            corpus = [word_to_id[word] for word in words]
        corpus_ar.append(corpus)
    del corpus
    return np.array(corpus_ar), word_to_id, id_to_word, token

In [4]:
corpus, word_to_id, id_to_word, token = preprocess(samples['reviews'])

C:\Users\master\anaconda3\envs\tf\lib\site-packages\ipykernel_launcher.py:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [11]:
# 길이 30
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

max_len = 30
X = pad_sequences(corpus, maxlen=max_len)
y= np.array(samples['ratings'])

In [12]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [13]:
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(random_state=42)
rnd_clf.fit(x_train, y_train)

y_pred_rf = rnd_clf.predict(x_test)

np.sum(y_test == y_pred_rf) / len(y_test)

0.684375